# Project - 4 Heart Disease Prediction - NN Model -Machine Learning

In [3]:
# Import dependencies
from pathlib import Path
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, balanced_accuracy_score,confusion_matrix,classification_report



## Step 1: Read the CVD_cleaned.csv data from the Resources folder into a Pandas DataFrame

In [4]:
# Read the csv file from the same folder 
data = Path("Resources/CVD_cleaned_data.csv")
df = pd.read_csv(data)
df=df.dropna()
df.head()

General_Health                  Checkup Exercise Heart_Disease Skin_Cancer  \
0           Poor  Within the past 2 years       No            No          No   
1      Very Good     Within the past year       No           Yes          No   
2      Very Good     Within the past year      Yes            No          No   
3           Poor     Within the past year      Yes           Yes          No   
4           Good     Within the past year       No            No          No   

  Other_Cancer Depression Diabetes Arthritis     Sex Age_Category  \
0           No         No       No       Yes  Female        70-74   
1           No         No      Yes        No  Female        70-74   
2           No         No      Yes        No  Female        60-64   
3           No         No      Yes        No    Male        75-79   
4           No         No       No        No    Male          80+   

   Height_(cm)  Weight_(kg)    BMI Smoking_History  Alcohol_Consumption  \
0        150.0        32.66  14.54             Yes                  0.0   
1        165.0        77.11  28.29              No                  0.0   
2        163.0        88.45  33.47              No                  4.0   
3        180.0        93.44  28.73              No                  0.0   
4        191.0        88.45  24.37             Yes                  0.0   

   Fruit_Consumption  Green_Vegetables_Consumption  FriedPotato_Consumption  
0               30.0                          16.0                     12.0  
1               30.0                           0.0                      4.0  
2               12.0                           3.0                     16.0  
3               30.0                          30.0                      8.0  
4                8.0                           4.0                      0.0

# Data Pre Processing

In [5]:
# To check the unique values in each column
df.nunique()

General_Health                     5
Checkup                            5
Exercise                           2
Heart_Disease                      2
Skin_Cancer                        2
Other_Cancer                       2
Depression                         2
Diabetes                           4
Arthritis                          2
Sex                                2
Age_Category                      13
Height_(cm)                       99
Weight_(kg)                      525
BMI                             3654
Smoking_History                    2
Alcohol_Consumption               31
Fruit_Consumption                 77
Green_Vegetables_Consumption      75
FriedPotato_Consumption           69
dtype: int64

In [6]:
#Checking for null Values
df.isnull().sum()

General_Health                  0
Checkup                         0
Exercise                        0
Heart_Disease                   0
Skin_Cancer                     0
Other_Cancer                    0
Depression                      0
Diabetes                        0
Arthritis                       0
Sex                             0
Age_Category                    0
Height_(cm)                     0
Weight_(kg)                     0
BMI                             0
Smoking_History                 0
Alcohol_Consumption             0
Fruit_Consumption               0
Green_Vegetables_Consumption    0
FriedPotato_Consumption         0
dtype: int64

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308854 entries, 0 to 308853
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   General_Health                308854 non-null  object 
 1   Checkup                       308854 non-null  object 
 2   Exercise                      308854 non-null  object 
 3   Heart_Disease                 308854 non-null  object 
 4   Skin_Cancer                   308854 non-null  object 
 5   Other_Cancer                  308854 non-null  object 
 6   Depression                    308854 non-null  object 
 7   Diabetes                      308854 non-null  object 
 8   Arthritis                     308854 non-null  object 
 9   Sex                           308854 non-null  object 
 10  Age_Category                  308854 non-null  object 
 11  Height_(cm)                   308854 non-null  float64
 12  Weight_(kg)                   308854 non-nul

In [8]:
# To Change the 'Exercise','Heart_Disease','Skin_Cancer','Other_Cancer','Depression','Diabetes','Arthritis' Columns 
#into numeric Values
df = df.replace({
    'Exercise': {'Yes': 1, 'No': 0},
    'Heart_Disease': {'Yes': 1, 'No': 0},
    'Skin_Cancer': {'Yes': 1, 'No': 0},
    'Other_Cancer': {'Yes': 1, 'No': 0},
    'Depression': {'Yes': 1, 'No': 0},
    'Diabetes': {'Yes': 1, 'No': 0,'No, pre-diabetes or borderline diabetes': 2,
                 'Yes, but female told only during pregnancy' : 3},
    'Arthritis': {'Yes': 1, 'No': 0},
    'Sex':{'Female' :1, 'Male' : 0},
    'Smoking_History' : {'Yes': 1, 'No': 0}
})

In [9]:
# Convert the 'General_Health' column to numerical values
# Define the mapping from categorical values to numerical values
category_mapping = {'Excellent': 4, 'Very Good': 3, 'Good': 2, 'Fair': 1, 'Poor': 0}

# Replace the values in the 'General_Health' column using the mapping
df['General_Health'] = df['General_Health'].replace(category_mapping)

In [10]:
# Convert the 'Checkup' column to numerical values
# Define the mapping from categorical values to numerical values
category1_mapping = {'Within the past year': 4, 'Within the past 2 years': 3, 'Within the past 5 years': 2, 
                    '5 or more years ago': 1, 'Never': 0}

# Replace the values in the 'General_Health' column using the mapping
df['Checkup'] = df['Checkup'].replace(category1_mapping)

In [11]:
# Convert the 'Age_Category' column to numerical values
def age_category(age):
    if age in ['18-24']:
        return 'Young'
    if age in ['25-29', '30-34', '35-39']:
        return 'Adult'
    elif age in ['40-44', '45-49', '50-54']:
        return 'Mid-Aged'
    elif age in ['55-59', '60-64','65-69']:
        return 'Senior-Adult'
    elif age in ['70-74', '75-79','80+']:
        return 'Elderly'
df['Age_Range'] = df['Age_Category'].apply(age_category)

age_mapping = {
    'Young': 0,
    'Adult': 1,
    'Mid-Aged': 2,
    'Senior-Adult': 3,
    'Elderly': 4
}

df['Age_Group'] = df['Age_Range'].map(age_mapping)

df.tail()

General_Health  Checkup  Exercise  Heart_Disease  Skin_Cancer  \
308849               3        4         1              0            0   
308850               1        2         1              0            0   
308851               3        1         1              0            0   
308852               3        4         1              0            0   
308853               4        4         1              0            0   

        Other_Cancer  Depression  Diabetes  Arthritis  Sex  ... Height_(cm)  \
308849             0           0         0          0    0  ...       168.0   
308850             0           0         1          0    0  ...       180.0   
308851             0           1         3          0    1  ...       157.0   
308852             0           0         0          0    0  ...       183.0   
308853             0           0         0          0    1  ...       160.0   

        Weight_(kg)    BMI  Smoking_History  Alcohol_Consumption  \
308849        81.65  29.05                0                  4.0   
308850        69.85  21.48                0                  8.0   
308851        61.23  24.69                1                  4.0   
308852        79.38  23.73                0                  3.0   
308853        81.19  31.71                0                  1.0   

        Fruit_Consumption  Green_Vegetables_Consumption  \
308849               30.0                           8.0   
308850               15.0                          60.0   
308851               40.0                           8.0   
308852               30.0                          12.0   
308853                5.0                          12.0   

        FriedPotato_Consumption     Age_Range Age_Group  
308849                      0.0         Adult         1  
308850                      4.0  Senior-Adult         3  
308851                      4.0         Adult         1  
308852                      0.0  Senior-Adult         3  
308853                      1.0      Mid-Aged         2  

[5 rows x 21 columns]

In [12]:
# Drop the 'Age_Range' column, since we serive the new column 'Age_Group'

df = df.drop(columns=['Age_Category', 'Age_Range'], axis=1)


df.tail()

General_Health  Checkup  Exercise  Heart_Disease  Skin_Cancer  \
308849               3        4         1              0            0   
308850               1        2         1              0            0   
308851               3        1         1              0            0   
308852               3        4         1              0            0   
308853               4        4         1              0            0   

        Other_Cancer  Depression  Diabetes  Arthritis  Sex  Height_(cm)  \
308849             0           0         0          0    0        168.0   
308850             0           0         1          0    0        180.0   
308851             0           1         3          0    1        157.0   
308852             0           0         0          0    0        183.0   
308853             0           0         0          0    1        160.0   

        Weight_(kg)    BMI  Smoking_History  Alcohol_Consumption  \
308849        81.65  29.05                0                  4.0   
308850        69.85  21.48                0                  8.0   
308851        61.23  24.69                1                  4.0   
308852        79.38  23.73                0                  3.0   
308853        81.19  31.71                0                  1.0   

        Fruit_Consumption  Green_Vegetables_Consumption  \
308849               30.0                           8.0   
308850               15.0                          60.0   
308851               40.0                           8.0   
308852               30.0                          12.0   
308853                5.0                          12.0   

        FriedPotato_Consumption  Age_Group  
308849                      0.0          1  
308850                      4.0          3  
308851                      4.0          1  
308852                      0.0          3  
308853                      1.0          2

In [13]:
# Import seaborn for the heat map
import seaborn as sns
import matplotlib.pyplot as plt
# calculate the correlation matrix on the numeric columns
corr = df.corr()
plt.figure(figsize=(12,10))
# plot the heatmap
sns.heatmap(corr,annot=True,cmap="magma",fmt='.2f')

<Axes: >

## Analysis:
### In the above Correlation Matrix with Heatmap we can observe that the none of the feature is  elated to the target variable. FriedPotato_Consumption very close to 0 indicates no correlation between the variables, in this case I'm trying to remove this column for optimization purpose. 

In [14]:
# df.corr() used to understand the relationships between different variables
df.corr()['Heart_Disease'].sort_values()[:-1]


General_Health                 -0.232503
Exercise                       -0.096347
Sex                            -0.072595
Alcohol_Consumption            -0.036569
Green_Vegetables_Consumption   -0.024043
Fruit_Consumption              -0.020055
FriedPotato_Consumption        -0.009227
Height_(cm)                     0.015780
Depression                      0.032526
BMI                             0.042666
Weight_(kg)                     0.045875
Checkup                         0.085596
Skin_Cancer                     0.090848
Other_Cancer                    0.092387
Smoking_History                 0.107797
Diabetes                        0.123101
Arthritis                       0.153913
Age_Group                       0.219765
Name: Heart_Disease, dtype: float64

In [15]:
# # Drop the 'FriedPotato_Consumption column, since that is very weak correlation with'Heart_Disease'

df = df.drop(columns=['FriedPotato_Consumption'], axis=1)


df.tail()

General_Health  Checkup  Exercise  Heart_Disease  Skin_Cancer  \
308849               3        4         1              0            0   
308850               1        2         1              0            0   
308851               3        1         1              0            0   
308852               3        4         1              0            0   
308853               4        4         1              0            0   

        Other_Cancer  Depression  Diabetes  Arthritis  Sex  Height_(cm)  \
308849             0           0         0          0    0        168.0   
308850             0           0         1          0    0        180.0   
308851             0           1         3          0    1        157.0   
308852             0           0         0          0    0        183.0   
308853             0           0         0          0    1        160.0   

        Weight_(kg)    BMI  Smoking_History  Alcohol_Consumption  \
308849        81.65  29.05                0                  4.0   
308850        69.85  21.48                0                  8.0   
308851        61.23  24.69                1                  4.0   
308852        79.38  23.73                0                  3.0   
308853        81.19  31.71                0                  1.0   

        Fruit_Consumption  Green_Vegetables_Consumption  Age_Group  
308849               30.0                           8.0          1  
308850               15.0                          60.0          3  
308851               40.0                           8.0          1  
308852               30.0                          12.0          3  
308853                5.0                          12.0          2

In [16]:
# Export the processed data to Resource Folder
df.to_csv('Resources/heart_cleaned_data.csv', index = False)

# Separate the Features (X) from the Target (y)
### Step 2: Create the labels set (`y`)  from the “Heart_Disease” column, and then create the features (`X`) DataFrame from the remaining columns.

In [17]:
y = df["Heart_Disease"]
X = df.drop(columns="Heart_Disease")

### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [18]:
y.value_counts()

Heart_Disease
0    283883
1     24971
Name: count, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=78
                                                    )
X_train.shape

(231640, 17)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

hidden_layer1 = 80
hidden_layer2 =30
input_features = len(X_train_scaled[0])

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_layer1, activation="relu", input_dim=input_features))
# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_layer2, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

C:\Users\tjosa\.conda\envs\dev\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 80)                  │           1,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 30)                  │           2,430 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              31 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,901 (15.24 KB)

 Trainable params: 3,901 (15.24 KB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9160 - loss: 0.2352
Epoch 2/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9182 - loss: 0.2259
Epoch 3/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.9199 - loss: 0.2215
Epoch 4/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9184 - loss: 0.2245
Epoch 5/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9190 - loss: 0.2228
Epoch 6/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.9186 - loss: 0.2233
Epoch 7/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9191 - loss: 0.2220
Epoch 8/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9195 - loss: 0.2209
Epoch 9/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9201 - loss: 0.2193
Epoch 10/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 11s 2ms/step - accuracy: 0.9204 - loss: 0.2190
Epoch 11/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9202 - loss: 0.2193
Epoch 12/100


7239/7239 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9209 - loss: 0.2127
Epoch 68/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 992us/step - accuracy: 0.9209 - loss: 0.2136
Epoch 69/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9213 - loss: 0.2141
Epoch 70/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9209 - loss: 0.2151
Epoch 71/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9217 - loss: 0.2133
Epoch 72/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9207 - loss: 0.2153
Epoch 73/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9205 - loss: 0.2162
Epoch 74/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9220 - loss: 0.2124
Epoch 75/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9220 - loss: 0.2121
Epoch 76/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9217 - loss: 0.2118
Epoch 77/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9215 - loss: 0.2143
Epoch 78/100
7239/723

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2413/2413 - 2s - 994us/step - accuracy: 0.9180 - loss: 0.2314
Loss: 0.2313869297504425, Accuracy: 0.9180330038070679


In [25]:
# Export our model to HDF5 file
nn_model.save('Resources/Heart.keras')

**Optimization attempt 1**



- Add more neurons to a hidden layer.(Add more nuurons for both layers)
- Add more hidden layers.


In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
hidden_layer1 = 100
hidden_layer2 =60
hidden_layer2 =20
input_features = len(X_train_scaled[0])

nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=hidden_layer1, activation="relu", input_dim=input_features))
# Second hidden layer
nn_model2.add(tf.keras.layers.Dense(units=hidden_layer2, activation="relu"))

# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()


# Compile the model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


# Train the model
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=100)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                      │ (None, 100)                 │           1,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 20)                  │           2,020 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (None, 1)                   │              21 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,841 (15.00 KB)

 Trainable params: 3,841 (15.00 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 998us/step - accuracy: 0.9158 - loss: 0.2374
Epoch 2/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9186 - loss: 0.2237
Epoch 3/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9181 - loss: 0.2246
Epoch 4/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9193 - loss: 0.2207
Epoch 5/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9188 - loss: 0.2222
Epoch 6/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9195 - loss: 0.2213
Epoch 7/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 900us/step - accuracy: 0.9196 - loss: 0.2218
Epoch 8/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 6s 891us/step - accuracy: 0.9190 - loss: 0.2213
Epoch 9/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9190 - loss: 0.2228
Epoch 10/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9200 - loss: 0.2201
Epoch 11/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 10s 1ms/step - accuracy: 0.9194 - loss: 0.2210
Epoch 12/100

7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 974us/step - accuracy: 0.9219 - loss: 0.2133
Epoch 67/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9212 - loss: 0.2140
Epoch 68/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9210 - loss: 0.2142
Epoch 69/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 962us/step - accuracy: 0.9220 - loss: 0.2121
Epoch 70/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 925us/step - accuracy: 0.9220 - loss: 0.2130
Epoch 71/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9217 - loss: 0.2122
Epoch 72/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 951us/step - accuracy: 0.9219 - loss: 0.2133
Epoch 73/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9216 - loss: 0.2138
Epoch 74/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 958us/step - accuracy: 0.9224 - loss: 0.2124
Epoch 75/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9219 - loss: 0.2121
Epoch 76/100
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 925us/step - accuracy: 0.9221 - loss: 0.2117
Epoch 77/10

In [28]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

2413/2413 - 2s - 971us/step - accuracy: 0.9183 - loss: 0.2329
Loss: 0.23291058838367462, Accuracy: 0.9182920455932617


**Optimization attempt 2**



- Use different activation functions for the hidden layers.
- Reduce the number of epochs to the training regimen.

In [29]:
hidden_layer1 = 80
hidden_layer2 =40
input_features = len(X_train_scaled[0])

nn_model3= tf.keras.models.Sequential()

# First hidden layer
nn_model3.add(tf.keras.layers.Dense(units=hidden_layer1, activation="sigmoid", input_dim=input_features))
# Second hidden layer
nn_model3.add(tf.keras.layers.Dense(units=hidden_layer2, activation="sigmoid"))

# Output layer
nn_model3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model3.summary()


# Compile the model
nn_model3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


# Train the model
fit_model = nn_model3.fit(X_train_scaled, y_train, epochs=10)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_7 (Dense)                      │ (None, 80)                  │           1,440 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 40)                  │           3,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 1)                   │              41 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,721 (18.44 KB)

 Trainable params: 4,721 (18.44 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step - accuracy: 0.9045 - loss: 0.2552
Epoch 2/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 6s 889us/step - accuracy: 0.9190 - loss: 0.2244
Epoch 3/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 989us/step - accuracy: 0.9188 - loss: 0.2241
Epoch 4/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 10s 944us/step - accuracy: 0.9181 - loss: 0.2243
Epoch 5/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.9194 - loss: 0.2228
Epoch 6/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 968us/step - accuracy: 0.9185 - loss: 0.2251
Epoch 7/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 925us/step - accuracy: 0.9205 - loss: 0.2208
Epoch 8/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 8s 1ms/step - accuracy: 0.9202 - loss: 0.2211
Epoch 9/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 961us/step - accuracy: 0.9189 - loss: 0.2224
Epoch 10/10
7239/7239 ━━━━━━━━━━━━━━━━━━━━ 7s 968us/step - accuracy: 0.9195 - loss: 0.2227


In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


2413/2413 - 2s - 848us/step - accuracy: 0.9203 - loss: 0.2224
Loss: 0.2224111258983612, Accuracy: 0.9202605485916138


### Analysis
First define the model and get Loss: 0.2313, Accuracy: 91.80%
These results indicate that the model achieved a high accuracy of around 91.83% and a relatively low loss of approximately 0.2282 on the training data. This suggests that the model has been optimized well and performs effectively on the training set.

To increase the performance of the model take extra steps to optimixe the model:
Step: 1 
•	Add more neurons to a hidden layer.(Add more neurons for both layers)
•	Add more hidden layers.
      After adding hidden layers and neurons  I have achieved accuracy  : 91.82% and the loss is 0.2329. Slight increase in the accuracy and decrese in the loss, which is good.

Step :2 
•	Used different activation functions for the hidden layers.
•	Reduce the number of epochs to the training regimen.
After adding hidden layers and neurons I have achieved accuracy  : 92.02% and the loss is 0.2224. Slight increase in the accuracy.

These results suggest that the model achieved a relatively low loss and a high accuracy on the training data, indicating that the model has been optimized effectively and performs well in making predictions on the training set.
